# Data simulation

This notebook demonstrates synthetic data creation with TomoPy to create sinograms with various types of defects:
1. Poisson noise
2. Rings
3. Zingers
4. Illumination drift
5. Incorrect center of rotation

In [ ]:
import tomopy
import matplotlib.pyplot as plt
import numpy as np

## Load a built-in object

TomoPy comes with 9 images which can be used a phantoms: baboon, barbara, cameraman, checkerboard, lena, peppers, phantom, shepp2d, shepp3d.

In [ ]:
obj = tomopy.peppers(size=128)
print(obj.shape) # tomopy phantoms are always 3D

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(obj[0])
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Sinogram simulation

TomoPy can simulate parallel-projection through a grid if given a list of angles.

In [ ]:
# provide angles in degrees but recieve radians
ang = tomopy.angles(nang=120, ang1=0, ang2=180)
print(ang)

In [ ]:
prj = tomopy.project(obj, ang, pad=True)

In [ ]:
print(prj.shape)
# Note that the result is padded by default

In [ ]:
print(prj.dtype)
# Note that the result a floating-point number instead of integer photon counts

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(prj[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

### Poisson Noise

In [ ]:
prj_poisson = tomopy.add_poisson(prj / 100) * 100

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(prj_poisson[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

### Rings

In [ ]:
def add_rings(tomo, std=0.05):
    """Add rings.
    
    Rings are caused by inconsistent pixel sensitivity across the detector.
    
    The sensitivity of the pixels is modeled as normally distributed with an
    average sensitivity of 1 and a standard deviation given.
    
    
    Parameters
    ----------
    tomo : ndarray
        3D tomographic data.
    std : float
        The standard deviation of the pixel sensitivity

    Returns
    -------
    ndarray
        Tomographic data with zingers added.
    """
    new_tomo = np.copy(tomo)
    sensitivity = np.random.normal(
        loc=1, scale=std,
        size=(1, new_tomo.shape[1], new_tomo.shape[2])
    )
    new_tomo = new_tomo * sensitivity
    return new_tomo

In [ ]:
prj_rings = add_rings(prj)

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(prj_rings[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

### Zingers

In [ ]:
def add_zingers(tomo, f=0.01, sat=2**16):
    """Add zingers.
    
    Zingers are caused by stray X-rays hitting the detector and causing pixels
    to saturate.
    
    The zingers are uniformly distributed across the data set with the given
    frequency.
    
    Parameters
    ----------
    tomo : ndarray
        3D tomographic data.
    f : float
        The fraction of measurements that are zingers.
    sat : float
        The pixel saturation value.

    Returns
    -------
    ndarray
        Tomographic data with zingers added.
    """
    zingers = np.random.uniform(0, 1, tomo.shape)
    zingers = zingers <= f  # five percent of measurements are zingers
    new_tomo = np.copy(tomo)
    new_tomo[zingers] = sat
    return new_tomo

In [ ]:
prj_zingers = add_zingers(prj, f=0.01, sat=2**13)

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(prj_zingers[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

### Illumination drift

In [ ]:
def add_drift(tomo, amp=0.2, period=50, mean=1):
    """Add illumination drift.
    
    Illumination drift is caused by the beam instability as the object rotates.
    
    This drift is modeled using a sinusoid. Which alters the illumination
    along the rotation dimension. The vertical dimension is constant.
    
    Parameters
    ----------
    tomo : ndarray
        3D tomographic data.
    amp : float
        The amplitude of the drift.
    period : float
        The period of the drift.

    Returns
    -------
    ndarray
        Tomographic data with zingers added.
    """
    new_tomo = np.copy(tomo)
    x = np.arange(tomo.shape[0])
    drift = amp * np.sin(2 * np.pi / period * x) + mean
    drift = drift + np.linspace(0, 1, len(x))
    drift = drift[:, np.newaxis, np.newaxis]
#     return drift + tomo * 0
    return drift * tomo

In [ ]:
prj_drift = add_drift(prj, amp=0.5)

In [ ]:
plt.figure(tight_layout=True)
plt.imshow(prj_drift[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

### Off-center

In [ ]:
prj_center = tomopy.project(obj, ang, pad=True, center=100)
print(prj.shape)
# Note that the result is padded by default

In [ ]:
plt.figure()
plt.imshow(prj_center[:, 0, :])
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.colorbar()
plt.show()

## Combine data sets and save to file

In [ ]:
labels = ['ideal', 'poisson', 'zingers', 'rings', 'illumination drift', 'off-center']
prj_all = np.concatenate([prj, prj_poisson, prj_zingers, prj_rings, prj_drift, prj_center], axis=1)

In [ ]:
def write_dxfile(fname, data, ang, data_white=None, data_dark=None, sample_description=''):
    """Store simulated data in data-exchange compliant format."""
    import dxfile.dxtomo as dx
    import os

    experimenter_name="Daniel Ching"
    experimenter_affiliation="Argonne National Laboratory" 
    experimenter_email="dching@anl.gov"
    instrument_comment="TomoPy Simlated Data"
    
    theta = ang / np.pi * 180
    
    if data_white is None:
        data_white = np.ones_like(data[0:1, ...])
    if data_dark is None:
        data_dark = np.zeros_like(data[0:1, ...])
    
    if (fname != None):
        if os.path.isfile(fname):
            print("Data Exchange file already exists: ", fname)
        else:
            # Create new folder.
            dirPath = os.path.dirname(fname)
            if not os.path.exists(dirPath):
                os.makedirs(dirPath)

            # Open DataExchange file
            f = dx.File(fname, mode='w')
            
            # Write the Data Exchange HDF5 file.
            f.add_entry(dx.Entry.experimenter(name={'value':experimenter_name}))
            f.add_entry(dx.Entry.experimenter(affiliation={'value':experimenter_affiliation}))
            f.add_entry(dx.Entry.experimenter(email={'value':experimenter_email}))
            f.add_entry(dx.Entry.instrument(comment={'value': instrument_comment}))
            f.add_entry(dx.Entry.sample( description={'value':sample_description}))    

            f.add_entry(dx.Entry.data(data={'value': data, 'units':'counts'}))
            f.add_entry(dx.Entry.data(data_white={'value': data_white, 'units':'counts'}))
            f.add_entry(dx.Entry.data(data_dark={'value': data_dark, 'units':'counts'}))
            f.add_entry(dx.Entry.data(theta={'value': theta, 'units':'degrees'}))

            f.close()
    else:
        raise ValueError("file name cannot be None")

In [ ]:
write_dxfile('data/data-simulated.h5', prj_all, ang,
            sample_description="Simulated data with various distortions.")

In [ ]:
## Reconstruct

In [ ]:
recon = tomopy.recon(tomo=prj_all, theta=ang, algorithm='gridrec')

In [ ]:
for z, label in zip(recon, labels):
    plt.figure(dpi=150)
    plt.title(label)
    plt.imshow(z)